In [2]:
from dotenv import load_dotenv

_ = load_dotenv()

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

/Users/kentdang/Projects/Disaster-Response/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [5]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [6]:
PLAN_PROMPT = """You are an expert disaster response planner tasked with creating a comprehensive \
    response plan for a given disaster scenario. Write such an outline for the user provided \
    disaster scenario. Give an outline of the plan along with any relevant notes or instructions \
    for the disaster response plan.
"""

In [7]:
RESPONDER_PROMPT = """You are a disaster response assistant tasked with executing a comprehensive disaster response plan. \
Utilize the detailed plan provided by the planning agent to implement the response effectively. Your responsibilities include \
coordinating resources, managing communication, and adapting to evolving situations. If there are updates or critiques from \
the planning agent or response teams, adjust your actions accordingly. Utilize all the information below as needed:

------

{content}"""

In [8]:
REFLECTION_PROMPT = """You are a disaster response expert reviewing a disaster response plan. 
Generate critique and recommendations for the provided plan. 
Provide detailed recommendations, including requests for additional details, improvements in strategy, and suggestions for better resource management and communication.
Ensure your feedback is constructive and actionable.

------

{content}"""

In [9]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can 
be used to enhance the following disaster response plan. Generate a list of search queries that will gather 
any relevant information. Only generate 3 queries max. 

------

{content}"""

In [10]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can 
be used to make any requested revisions to the disaster response plan (as outlined below). 
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max. 

------

{content}"""

In [11]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [12]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [13]:
def plan_node(state: AgentState):
    message = [ 
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(message)
    return {'plan':response.content}

In [14]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [15]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=REFLECTION_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [16]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [17]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [18]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [19]:
builder = StateGraph(AgentState)

In [20]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [21]:
builder.set_entry_point("planner")

In [22]:
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)


In [23]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [24]:
graph = builder.compile(checkpointer=memory)

In [25]:
thread = {"configurable": {"thread_id": "1"}}
# for s in graph.stream({
#     'task': "Floods and Landslides in Bangladesh",
#     "max_revisions": 2,
#     "revision_number": 1,
# }, thread):
#     print(s)

{'planner': {'plan': '**Disaster Response Plan: Floods and Landslides in Bangladesh**\n\n**I. Pre-Disaster Phase: Preparedness**\n\n1. **Risk Assessment and Planning**\n   - Identify high-risk areas prone to floods and landslides.\n   - Develop evacuation routes and safe shelters.\n   - Establish early warning systems and communication channels.\n\n2. **Community Awareness and Training**\n   - Conduct regular drills and training sessions for community members.\n   - Educate the population on safety measures and evacuation procedures.\n   - Distribute informational materials in local languages.\n\n3. **Resource Stockpiling**\n   - Pre-position emergency supplies like food, water, medical supplies, and blankets.\n   - Ensure availability of rescue equipment such as boats, life jackets, and ropes.\n   - Coordinate with local authorities and NGOs for additional support.\n\n**II. During Disaster Phase: Response**\n\n1. **Early Warning and Activation**\n   - Monitor weather forecasts and riv

In [29]:
def get_disaster_response_plan(task: str, max_revisions: int):
    state = {
        'task': task,
        'plan': '',
        'draft': '',
        'critique': '',
        'content': [],
        'revision_number': 1,
        'max_revisions': max_revisions,
    }
    
    # Process the state graph
    for s in graph.stream(state, thread):
        result = s

    return result